In [1]:
# 1. Import dependencies
import requests
import json
import time
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import h3

from space2stats_client import Space2StatsClient

In [2]:
# 2. Configure test parameters
API_URL = "https://space2stats.ds.io"  # Replace with your actual API endpoint
LOCAL_API_URL = "http://127.0.0.1:8000"  # Replace with your actual API endpoint
DEV_API_URL = "https://fcz9g668q1.execute-api.us-east-1.amazonaws.com"  # Replace with your actual API endpoint

client_prod = Space2StatsClient()
client_local = Space2StatsClient(LOCAL_API_URL)
client_dev = Space2StatsClient(DEV_API_URL)

In [3]:
# 1) Install the library (and shapely if you don't already have it)
#    !pip install h3ronpy shapely

from shapely.geometry import Polygon
from h3ronpy.vector import geometry_to_cells
from h3ronpy import cells_to_string

# 2) Define your Central Africa bounding‐box (lon, lat)
min_lon, min_lat = -10.0, -20.0
max_lon, max_lat =  40.0,  20.0

bbox = Polygon([
    (min_lon, min_lat),
    (max_lon, min_lat),
    (max_lon, max_lat),
    (min_lon, max_lat),
    (min_lon, min_lat),
])

# 3) Choose resolution
resolution = 6

# 4) Fill the polygon with H3 cells (returns an Array<UInt64>)
num_cells = geometry_to_cells(bbox, resolution)

# 5) Convert numeric H3 indexes to hex‐string IDs
hex_ids = cells_to_string(num_cells)  # → Array[str]

# 6) (Optional) turn into a plain Python list and inspect
hex_list = hex_ids.to_pylist()
print(f"Found {len(hex_list)} hexagons at res={resolution}")
print(hex_list[:10000])
print(len(hex_list))
sample_hex_ids = hex_list

Found 683169 hexagons at res=6
['863e82087ffffff', '863e8208fffffff', '863e82097ffffff', '863e8209fffffff', '863e820c7ffffff', '863e820cfffffff', '863e820d7ffffff', '863e820dfffffff', '863e82207ffffff', '863e8220fffffff', '863e82217ffffff', '863e8221fffffff', '863e82247ffffff', '863e8224fffffff', '863e82257ffffff', '863e8225fffffff', '863e82287ffffff', '863e8228fffffff', '863e82297ffffff', '863e8229fffffff', '863e822a7ffffff', '863e822afffffff', '863e822b7ffffff', '863e822c7ffffff', '863e822cfffffff', '863e822d7ffffff', '863e822dfffffff', '863e822e7ffffff', '863e822efffffff', '863e822f7ffffff', '863e82387ffffff', '863e8238fffffff', '863e82397ffffff', '863e8239fffffff', '863e82407ffffff', '863e8240fffffff', '863e82417ffffff', '863e8241fffffff', '863e82427ffffff', '863e8242fffffff', '863e82437ffffff', '863e82447ffffff', '863e8244fffffff', '863e82457ffffff', '863e8245fffffff', '863e82467ffffff', '863e8246fffffff', '863e82477ffffff', '863e82487ffffff', '863e8248fffffff', '863e82497ffffff',

In [4]:
ISO3 = "BRA" # Sri Lanka 
ADM = "ADM0" # Level 2 administrative boundaries
adm_boundaries = client_dev.fetch_admin_boundaries(ISO3, ADM)

# Tests on the Dev Server

In [9]:
all_fields = client_dev.get_fields()

### Test #1: Client Test (413)

In [12]:
# Client method
start_time = time.time()
client_dev.get_summary_by_hexids(
    sample_hex_ids,
    all_fields[2:67],
)
end_time = time.time()
response_time = end_time - start_time
print(f"{response_time:.2f}s")

Fetching data for 683169 hex IDs...


Exception: Failed to get_summary_by_hexids (HTTP 413): Request Entity Too Large: The request payload exceeds API Gateway limits

Hint: Try again with a smaller request or making multiple requests with smaller payloads. The factors to consider are the number of hexIds (ie. AOI), the number of fields requested, and the date range (if timeseries is requested).

### Test #2: API Endpoint Test (413)

In [21]:
# API endpoint

# Request payload
payload = {
    "hex_ids": sample_hex_ids,  # Your hex IDs
    "fields":  all_fields[2:67],  # Fields you want
    "geometry": "polygon"  # Optional: "polygon", "point", or None
}

response = requests.post(
    f"{API_URL}/summary_by_hexids",
    json=payload,
    headers={"Content-Type": "application/json"}
)

print("Raw response:")

print(response.text)
print(response)

Raw response:
{"message":"Request Entity Too Large"}
<Response [413]>


### Test #3: Client Test (503)

In [20]:
# Client method
start_time = time.time()
spatial_join_method = "touches"
res = client_dev.get_summary(
    adm_boundaries,
    spatial_join_method,
    all_fields[:60],
)
end_time = time.time()
response_time = end_time - start_time
print(f"{response_time:.2f}s")

Fetching data for boundary 1 of 1...


Exception: Failed to get_summary (HTTP 500): Database connection failed: out of memory
DETAIL:  Failed on request of size 40 in memory context "MessageContext".

### Test #4: API Endpoint Test (503)

In [19]:
# For spatial queries with boundaries
payload = {
    "aoi": {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [-10.0, -20.0],
                [40.0, -20.0],
                [40.0, 20.0],
                [-10.0, 20.0],
                [-10.0, -20.0]
            ]]
        }
    },
    "spatial_join_method": "touches",  # or "intersects", "within"
    "fields":  all_fields[2:67],  # Fields you want
    "geometry": "polygon"  # Optional
}

response = requests.post(
    f"{DEV_API_URL}/summary",
    json=payload,
    headers={"Content-Type": "application/json"}
)
response
print("Raw response:")
print(response.text)

Raw response:
{"error":"Database connection failed","detail":"out of memory\nDETAIL:  Failed on request of size 13700924 in memory context \"MessageContext\"."}
